In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.fftpack import idct

%matplotlib inline
%config InlineBackend.figure_format='retina'

Variable initialization


In [ ]:
M = 32  # signal dimension
N = 2 * M  # number of atoms in the expansion

C = np.zeros(
    (M, M)
)  # matrix containing the standard basis (a kronecker delta in each column)
DCT = np.zeros(
    (M, M)
)  # matrix containing the DCT basis (a DCT function in each column)

Generate the 1D-DCT basis


In [ ]:
for i in range(M):
    DCT[:, i] = idct(np.eye(M)[:, i], type=2, norm="ortho")  # DCT basis

Generating the 1-D canonical basis


In [ ]:
for i in range(M):
    C[:, i] = np.eye(M)[:, i]

Define the dictionary $D = [DCT, C]$


In [ ]:
D = np.hstack((DCT, C))

plt.figure(figsize=(10, 10))
plt.imshow(D)
plt.title(f"Our dictionary M = {M}, N = {N}")

## Generate a signal that is sparse w.r.t. $D$

To this purpose add a spike to the sum of few DCT atoms, i.e., add a spike to $\mathbf{s}$ that is sparse w.r.t. $C$. Bear in mind that the spike is to be considered a signal to be reconstructed, rather than noise


In [ ]:
L = 4
sigma_noise = 0.2

Randomly define the coefficients of a sparse representation w.r.t. $DCT$ (make sure the nonzero coefficients are sufficiently large)


In [ ]:
x0 = np.zeros(N)
nonzero_idx = np.random.choice(
    M, L, replace=False
)  # choose L unique indices in DCT part
x0[nonzero_idx] = np.random.randn(L) * 2 + np.random.choice([-1, 1], L) * 3

Choose spike location


In [ ]:
spikeLocation = np.random.randint(M, N)
x0[spikeLocation] += -10

Synthetize the corresponding signal in the signal domain and add noise


In [ ]:
s0 = D @ x0
s = s0 + sigma_noise * np.random.normal(scale=2, size=M)

Plot the sparse signal


In [ ]:
LN_WDT = 2
MRK_SZ = 10

plt.figure(figsize=(6, 6))
plt.plot(s0, "b--", linewidth=LN_WDT + 1)
plt.plot(s, "r--x", linewidth=LN_WDT - 1)
plt.title(f"Sparse signal in DCT domain (L = {L:.0f})")
plt.legend(["original", "noisy"])


## Orthogonal Matching Pursuit

Initialize all the variables, including the residual, namely the components of the signals that can not be represented (here the signal at the very beginning)


In [ ]:
x_OMP = np.zeros(N)

# residual
r = s.copy()

# support set
omega = []

MINIMUM_NORM_RES = 0.1

OMP loop starts.

Stoppint criteria: continue until the sparsity of the representation reaches L


In [ ]:
while np.count_nonzero(x_OMP) < L and np.linalg.norm(r) > MINIMUM_NORM_RES:
    # SWEEP STEP: look for the column of D that matches at best noisySignal
    # compute the residual w.r.t. each column of D
    e = np.zeros(N)
    for j in range(N):
        e[j] = D[:, j].T @ r

    # find the column of D that matches at best r
    jStar = np.argmax(np.abs(e))

    # UPDATE the support set with the jStar coefficient
    omega.append(jStar)

    # update the coefficients by solving the least square problem min ||D_omega x - s ||
    D_omega = D[:, omega]
    x_omega = np.linalg.lstsq(D_omega, s, rcond=None)[0]
    x_OMP = np.zeros(N)
    x_OMP[omega] = x_omega

    # update the residual
    r = s - D_omega @ x_omega

SYNTHESIS: reconstruct the signal, by inverting the transformation to reconstruct the signal


In [ ]:
s_hat_OMP = D @ x_OMP

Show the result


In [ ]:
LN_WDT = 2
MRK_SZ = 10

fix, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].plot(s0, "b-o", linewidth=LN_WDT + 1)
ax[0].plot(s, "r-x", linewidth=LN_WDT - 1)
ax[0].plot(s_hat_OMP, "m-", linewidth=LN_WDT)
ax[0].set_title(f"Sparse signal w.r.t D (L = {L:.0f})")
ax[0].legend(["original (s0)", "noisy (s)", "MP estimate"])

ax[1].stem(x0, linefmt="b-", markerfmt="C0o")
ax[1].stem(x_OMP, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("Coefficients")
ax[1].legend(["coefficients of s0 (x0)", "coefficients of s_hat (x_hat)"])

## Least Squares Orthogonal Matching Pursuit

Initialize all the variables, including the residual, namely the components of the signals that can not be represented (here the signal at the very beginning)


In [ ]:
x_LSOMP = np.zeros(N)

# residual
r = s.copy()

# support set
omega = []

MINIMUM_NORM_RES = 0.1

LSOMP loop starts.

Stoppint criteria: continue until the sparsity of the representation reaches L


In [ ]:
while np.count_nonzero(x_LSOMP) < L and np.linalg.norm(r) > MINIMUM_NORM_RES:
    # SWEEP STEP: find the best column by solving the LS problem
    if len(omega) == 0:
        # at the first iteration perform the usual sweep step
        e = np.zeros(N)
        for j in range(N):
            e[j] = np.linalg.norm(r - np.dot(D[:, j], r) * D[:, j]) ** 2
        jStar = np.argmin(e)
        omega.append(jStar)
        x_LSOMP = np.zeros(N)
        x_LSOMP[jStar] = np.dot(D[:, jStar], s)
    else:
        # perform the sweep step by solving the LS problem
        best_residual = np.inf
        jStar = -1

        for j in range(N):
            if j not in omega:  # only consider atoms not already in support
                # Try adding atom j to current support
                temp_omega = omega + [j]
                D_temp = D[:, temp_omega]
                # Solve LS problem for this expanded support
                x_temp = np.linalg.lstsq(D_temp, s, rcond=None)[0]
                # Compute residual
                residual_norm = np.linalg.norm(s - D_temp @ x_temp) ** 2

                if residual_norm < best_residual:
                    best_residual = residual_norm
                    jStar = j

        # UPDATE the support set with the jStar coefficient
        omega.append(jStar)

        # update the coefficients by solving LS over full support
        D_omega = D[:, omega]
        x_omega = np.linalg.lstsq(D_omega, s, rcond=None)[0]
        x_LSOMP = np.zeros(N)
        x_LSOMP[omega] = x_omega

    # update the residual
    D_omega = D[:, omega]
    r = s - D_omega @ x_LSOMP[omega]

SYNTHESIS: reconstruct the signal, by inverting the transformation to reconstruct the signal


In [ ]:
s_hat_LSOMP = D @ x_LSOMP

Show the result


In [ ]:
LN_WDT = 2
MRK_SZ = 10

fix, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].plot(s0, "b-o", linewidth=LN_WDT + 1)
ax[0].plot(s, "r-x", linewidth=LN_WDT - 1)
ax[0].plot(s_hat_LSOMP, "m-", linewidth=LN_WDT)
ax[0].set_title(f"Sparse signal w.r.t D (L = {L:.0f})")
ax[0].legend(["original (s0)", "noisy (s)", "MP estimate"])

ax[1].stem(x0, linefmt="b-", markerfmt="C0o")
ax[1].stem(x_LSOMP, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("Coefficients")
ax[1].legend(["coefficients of s0 (x0)", "coefficients of s_hat (x_hat)"])